In [1]:
%matplotlib inline

In [2]:
import datetime, time
import lxml

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [46]:
seqs = pd.read_csv("../data/sequential_id.csv")

In [47]:
seqs['name'] = seqs.name.str.upper()
seqs['name'] = seqs.name.replace('AURÉLIO NOMURA', 'AURELIO NOMURA')
seqs['name'] = seqs.name.replace('CAIO MIRANDA CARNEIRO', 'CAIO MIRANDA')
seqs['name'] = seqs.name.replace('ANTONIO DONATO', 'DONATO')
seqs['name'] = seqs.name.replace('EDUARDO SUPLICY', 'EDUARDO MATARAZZO SUPLICY') 
seqs['name'] = seqs.name.replace('MARIO COVAS NETO', 'MÁRIO COVAS NETO')
seqs['name'] = seqs.name.replace('REGINALDO TRIPOLI', 'REGINALDO TRÍPOLI')
seqs['name'] = seqs.name.replace('ZÉ TURIN', 'ZÉ TURIM')

In [48]:
seqs.head()

,sequential_id,name
0,250000011625,ABOU ANNI
1,250000019125,ADILSON AMADEU
2,250000004979,ADRIANA RAMALHO
3,250000022074,ALFREDINHO
4,250000004981,ALINE CARDOSO


In [49]:
xmls = [] 
xmls.append('http://www2.camara.sp.gov.br/sisgv/Arquivos/201701.XML')

In [50]:
lista_gastos=[]
for link in xmls:
    r = requests.get(link)
    xml = BeautifulSoup(r.content, 'xml')
    gastos = xml.find_all('TabelaPortalITEMREEMBOLSO')
    for s in gastos:
        chave = s.Chave.text
        centro_custos = s.CENTROCUSTOSID.text
        departamento = s.DEPARTAMENTO.text
        tipodep = s.TIPODEDEPARTAMENTO
        vereador = s.VEREADOR.text
        ano = s.ANO.text
        mes = s.MES.text
        despesa = s.DESPESA.text
        cnpj = s.CNPJ.text
        fornecedor = s.FORNECEDOR.text
        valor = s.VALOR.text
        data = datetime.date(int(ano), int(mes), 1)

        dict_gastos = {'DATA': data, 'Chave':chave, 'CENTROCUSTOSID': centro_custos, 'DEPARTAMENTO': departamento, 
                'TIPO_DEPARTAMENTO':tipodep, 'VEREADOR': vereador, 'DESPESA':despesa, 'CNPJ': cnpj, 
                       'FORNECEDOR': fornecedor, 'VALOR': valor}
        lista_gastos.append(dict_gastos)

In [51]:
df = pd.DataFrame().from_records(lista_gastos)

In [52]:
df = df[['DATA', 'VEREADOR', 'DEPARTAMENTO', 'DESPESA', 'VALOR', 'FORNECEDOR','CNPJ',]]
df[['DATA']] = df[['DATA']].apply(pd.to_datetime)
df[['VALOR']] = df[['VALOR']].apply(pd.to_numeric)

In [53]:
df = pd.merge(df, seqs, how='left', left_on='VEREADOR', right_on='name')

In [56]:
df = df.drop(['name'], axis=1)

In [59]:
df = df.rename(columns={
    'DATA': 'date', 
    'VEREADOR': 'candidate',
    'DEPARTAMENTO': 'department', 
    'DESPESA': 'expense',
    'VALOR': 'value', 
    'FORNECEDOR': 'provider', 
    'CNPJ': 'cnpj',
})

In [60]:
df

,date,candidate,department,expense,value,provider,cnpj,sequential_id
0,2017-01-01,ABOU ANNI,GABINETE DE VEREADOR,COMBUSTIVEL,880.00,PROTOTIPO AUTO POSTO LTDA.,43.900.851/0001-01,2.500000e+11
1,2017-01-01,ABOU ANNI,GABINETE DE VEREADOR,INTERMEDIADO - CORREIOS,5993.27,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
2,2017-01-01,ABOU ANNI,GABINETE DE VEREADOR,MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE C...,219.45,CENTER PAPEIS COMERCIAL LTDA.,06.226.820/0001-82,2.500000e+11
3,2017-01-01,ADILSON AMADEU,GABINETE DE VEREADOR,ELABORAÇÃO/MANUTENÇAO DE SITE/HOSPEDAGEM,5946.00,GENERICAPONTOCOM SERVIÇOS DE INTERNET LTDA,11.913.587/0001-46,2.500000e+11
4,2017-01-01,ADILSON AMADEU,GABINETE DE VEREADOR,INTERMEDIADO - CORREIOS,1892.84,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
5,2017-01-01,ADILSON AMADEU,GABINETE DE VEREADOR,INTERMEDIADO - LOCAÇÃO DE VEÍCULOS,2058.08,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
6,2017-01-01,ADRIANA RAMALHO,GABINETE DE VEREADOR,COMBUSTIVEL,901.00,POSTO JENNER LTDA,61.699.179/0001-01,2.500000e+11
7,2017-01-01,ADRIANA RAMALHO,GABINETE DE VEREADOR,INTERMEDIADO - CORREIOS,15.30,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
8,2017-01-01,ADRIANA RAMALHO,GABINETE DE VEREADOR,INTERMEDIADO - LOCAÇÃO DE VEÍCULOS,1330.74,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
9,2017-01-01,ADRIANA RAMALHO,GABINETE DE VEREADOR,MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE C...,1985.85,CENTER PAPEIS COMERCIAL LTDA.,06.226.820/0001-82,2.500000e+11


In [61]:
df.to_csv("../data/expenses.csv", index=False)